In [56]:
import pandas as pd
import pandasql as ps
import plotly.express as px

In [57]:
crime_data=pd.read_csv('LA Crime 2020-Present.csv')
crime_data.rename(columns={'Date Rptd':'Date'},inplace=True)
crime_data[['Month','Day','Year']]=crime_data.Date.str.split('/',expand=True)

crime_data[['Year_EXP','Time','AM_PM']]=crime_data.Year.str.split(' ',expand=True)
crime_data.drop(columns=['Date','Year'],inplace=True)
crime_data.rename(columns={'Year_EXP':'Year'},inplace=True)
crime_data

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Month,Day,Year,Time,AM_PM
0,190326475,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,...,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,03,01,2020,12:00:00,AM
1,200106753,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,...,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,02,09,2020,12:00:00,AM
2,200320258,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,...,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,11,11,2020,12:00:00,AM
3,200907217,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,...,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,05,10,2023,12:00:00,AM
4,220614831,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,...,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,08,18,2022,12:00:00,AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986495,242011172,08/17/2024 12:00:00 AM,2300,20,Olympic,2033,1,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0344 1822,...,NaN,3700 WILSHIRE BL,NaN,34.0617,-118.3066,08,20,2024,12:00:00,AM
986496,240710284,07/23/2024 12:00:00 AM,1400,7,Wilshire,788,1,510,VEHICLE - STOLEN,NaN,...,NaN,4000 W 23RD ST,NaN,34.0362,-118.3284,07,24,2024,12:00:00,AM
986497,240104953,01/15/2024 12:00:00 AM,100,1,Central,101,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 0400 0416,...,NaN,1300 W SUNSET BL,NaN,34.0685,-118.2460,01,15,2024,12:00:00,AM
986498,240309674,04/24/2024 12:00:00 AM,1500,3,Southwest,358,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",1822 0334 0416 0445 0449 1202,...,NaN,FLOWER ST,JEFFERSON BL,34.0215,-118.2868,04,24,2024,12:00:00,AM


In [58]:
query="""
SELECT  Year,Month, COUNT(*) AS 'Crime Count'
FROM crime_data
GROUP BY 1,2
"""
crime_count=ps.sqldf(query)
crime_count

,Year,Month,Crime Count
0,2020,01,16060
1,2020,02,16490
2,2020,03,15539
3,2020,04,15179
4,2020,05,16331
5,2020,06,16955
6,2020,07,16907
7,2020,08,16605
8,2020,09,15636
9,2020,10,16142


In [59]:
query="""
SELECT 
  (CASE 
    WHEN AM_PM LIKE 'AM%' THEN 'AM'
    ELSE 'PM'
  END) AS Time, 
  COUNT(*) AS `Crime Count`
FROM crime_data
GROUP BY Time
ORDER BY `Crime Count` DESC;
"""
names_counts=ps.sqldf(query)
names_counts

,Time,Crime Count
0,AM,986500


In [60]:
query="""
SELECT Time,COUNT(*)
FROM crime_data
GROUP BY 1;
"""
tim_count=ps.sqldf(query)
tim_count

,Time,COUNT(*)
0,12:00:00,986500


### Now it seems we have one specific time of day everything is recorded and for that we will drop everything related to the time

In [61]:
crime_data.drop(columns=['DATE OCC','Time','AM_PM'],inplace=True)


### Counting the Percentage of male to female crimes 

In [62]:
query="""
SELECT (
    CASE
    WHEN `Vict Sex` like '%M%' THEN 'M'
    WHEN `Vict Sex` like '%F%' THEN 'F'
    ELSE 'Un-Known'
    END
 ) AS Gender, COUNT(*) AS 'Crime Exposure'
FROM crime_data
GROUP BY 1;
"""
sex_t_vict=ps.sqldf(query)
sex_t_vict

,Gender,Crime Exposure
0,F,353688
1,M,397948
2,Un-Known,234864


In [63]:
query="""
Select `Crm Cd Desc`, Year, Count(*) as 'Occurrence count'
FROM crime_data
GROUP BY 1,2;
"""
crime_desc=ps.sqldf(query)
crime_desc

,Crm Cd Desc,Year,Occurrence count
0,ARSON,2020,660
1,ARSON,2021,630
2,ARSON,2022,530
3,ARSON,2023,570
4,ARSON,2024,166
...,...,...,...
655,WEAPONS POSSESSION/BOMBING,2020,9
656,WEAPONS POSSESSION/BOMBING,2021,10
657,WEAPONS POSSESSION/BOMBING,2022,6
658,WEAPONS POSSESSION/BOMBING,2023,11


In [64]:
query="""
Select `Crm Cd Desc`, Count(*) as 'Occurrence count'
FROM crime_data
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 5;
"""
desc_count=ps.sqldf(query)
desc_count

,Crm Cd Desc,Occurrence count
0,VEHICLE - STOLEN,111632
1,BATTERY - SIMPLE ASSAULT,74717
2,BURGLARY FROM VEHICLE,61633
3,THEFT OF IDENTITY,61148
4,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",59848


In [65]:
fig = px.bar(desc_count, 
             x='Occurrence count', 
             y="Crm Cd Desc", 
             title="Top 5 Crime Categories by Occurrence",
             labels={"Crm Cd Desc": "Crime Category", "Occurrence count": "Number of Occurrences"},
             orientation='h')

# Optionally resize the figure for better spacing (optional)
fig.update_layout(width=900, height=500)

fig.show()


In [67]:
query="""
Select `AREA NAME`, Count(*) as 'Occurrence count', `Year`
FROM crime_data
GROUP BY 1,3
ORDER BY 2 DESC 
"""
area_count=ps.sqldf(query)
area_count

,AREA NAME,Occurrence count,Year
0,Central,17694,2022
1,Central,17081,2023
2,77th Street,14614,2022
3,77th Street,14151,2023
4,Pacific,13862,2023
...,...,...,...
100,Newton,4862,2024
101,Harbor,4740,2024
102,Northeast,4618,2024
103,Hollenbeck,3754,2024


In [72]:
query="""
SELECT `Year`||'/'||`Month` AS 'Time',COUNT(*) AS 'Occurrence Count'
FROM crime_data
GROUP BY 1
ORDER BY 1,2;
"""
desc_count=ps.sqldf(query)
desc_count.drop(desc_count.tail(7).index,inplace=True)
desc_count

,Time,Occurrence Count
0,2020/01,16060
1,2020/02,16490
2,2020/03,15539
3,2020/04,15179
4,2020/05,16331
5,2020/06,16955
6,2020/07,16907
7,2020/08,16605
8,2020/09,15636
9,2020/10,16142


In [73]:
fig=px.line(desc_count,x='Time',y='Occurrence Count',title='Crime Count Over Time')
fig.show()

In [79]:
query="""
SELECT `Weapon Desc`, COUNT (*) AS 'Occurrence Count'
FROM crime_data
WHERE `Weapon Desc` is not 'UNKNOWN WEAPON/OTHER WEAPON' and `Weapon Desc` is not NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;
"""
weapon_count=ps.sqldf(query)
weapon_count

,Weapon Desc,Occurrence Count
0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",174484
1,VERBAL THREAT,23806
2,HAND GUN,20123
3,SEMI-AUTOMATIC PISTOL,7250
4,KNIFE WITH BLADE 6INCHES OR LESS,6826


In [80]:
fig = px.bar(weapon_count,x='Occurrence Count',y='Weapon Desc',title='Weapons Usage',orientation='h')
fig.update_layout(width=900, height=500)
fig.show()

In [82]:
query="""
SELECT 
    CASE 
        WHEN `Crm Cd Desc` LIKE '%HOMICIDE%' THEN 'HOMICIDE'
        ELSE 'Other'
    END AS Crime, COUNT(*)
FROM crime_data
WHERE Crime = 'HOMICIDE'
GROUP BY 1
"""
killing=ps.sqldf(query)
killing

,Crime,COUNT(*)
0,HOMICIDE,1566


In [86]:
query="""
SELECT (
    CASE
    WHEN `Vict Age` <=18 THEN '18 and Under'
    WHEN `Vict Age` >18 AND`Vict Age` <31  THEN '19-30'
    WHEN `Vict Age` >30 AND`Vict Age` <41  THEN '31-40'
    WHEN `Vict Age` >40 AND`Vict Age` <51  THEN '41-50'
    ELSE '51+'
END ) AS 'Victim Age', COUNT(*) AS 'Assult Count'
FROM crime_data
GROUP BY 1
ORDER BY 2 DESC;
"""
vict_age=ps.sqldf(query)
vict_age

,Victim Age,Assult Count
0,18 and Under,292984
1,19-30,208571
2,31-40,185472
3,51+,174377
4,41-50,125096
